In [477]:
import csv
from math import *
import numpy as np
from matplotlib import pyplot as plt
from input_generation import *
import time

In [478]:
VERBOSE = False

In [479]:
IDENTICAL = -1
VERTICAL = 0
HORIZONTAL = 1
OBLIQUE = 2
T_l = 30 #tri nguong~ dung de chia khoang (T_l trong crack report)
TEST_TIMES = 50
# CSV_NAME = "./Horizontal/10.csv"
CSV_NAME = "./Horizontal/19.csv"
LOG =  open("log.txt","a+")

In [480]:
def parse_csv(FILE):
    point_list = []
    with open(FILE) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        length = -1
        for row in csv_reader:
            if length == -1:
                length += 1
            else:
                point_list.append((float(row[0]), float(row[1])))
                length += 1
    return point_list

In [481]:
 def in_range(value, range):
    if range[0] <= value and value <= range[1]:
        return True
    if range[1] <= value and value <= range[0]:
        return True
    return False

In [482]:
def group_to_pairs(x_coordinates, y_coordinates, length):
        def relative_position(A, B):
            if A[0] != B[0] and A[1] != B[1]:
                return OBLIQUE
            elif A[0] == B[0] and A[1] != B[1]:
                return VERTICAL
            elif A[0] != B[0] and A[1] == B[1]:
                return HORIZONTAL
            else:
                return IDENTICAL
        all_pairs_position = {
            "VERTICAL": {},
            "HORIZONTAL": {}
        }
        for i in range(0, length - 1):
            position = relative_position(
                (x_coordinates[i], y_coordinates[i]), (x_coordinates[i+1], y_coordinates[i+1]))
            if position == IDENTICAL or position == OBLIQUE:
                continue
            if position == VERTICAL:
                previous_value = all_pairs_position.get('VERTICAL').get(x_coordinates[i])
                update_value = [(y_coordinates[i], y_coordinates[i+1])]
                if previous_value != None:
                    update_value = previous_value + update_value
                all_pairs_position['VERTICAL'][x_coordinates[i]] = update_value
            if position == HORIZONTAL:
                previous_value = all_pairs_position.get('HORIZONTAL').get(y_coordinates[i])
                update_value = [(x_coordinates[i], x_coordinates[i+1])]
                if previous_value != None:
                    update_value = previous_value + update_value
                all_pairs_position['HORIZONTAL'][y_coordinates[i]] = update_value
        return all_pairs_position

In [483]:
#Grouping points
point_list = parse_csv(CSV_NAME)
#print(point_list)
# Testcase
# print(isCollide(point_list, (12.4, 13), (12.4, 13.7)))
# print('------------------')
# print(isCollide(point_list, (17, 3.5), (18, 3.5)))
# print('------------------')

In [484]:
def find_endpoint(x_value, all_pairs_position):
    endpoint = {'y':{},'x':{}}
    for point, ran in all_pairs_position['HORIZONTAL'].items():
        for sub_ran in ran:
            if in_range(x_value, sub_ran):
                endpoint['y'][point] = sub_ran
    for point, ran in all_pairs_position['VERTICAL'].items():
        if x_value == point:
            endpoint['x'][point] = ran
    return endpoint

In [485]:
def distance(A, B):
    return sqrt((A[0] - B[0])**2 + (A[1] - B[1])**2)

In [486]:
def findLine(Origin, Angle):
    if Angle == 90:
        print("Invalid. Angle is 90!")
        return ('None', 'None')
    #find slope
    a = tan(radians(Angle))
    # Line y = a*x + b
    b = (Origin[1] - a*Origin[0])
    return (a,b)

In [487]:
LOG.write("Test: " + CSV_NAME + "\n")
LOG.write("Number of loops run:{}\n".format(TEST_TIMES))

23

In [469]:
start = time.time()
for test_times in range(TEST_TIMES):
    x_coordinates = []
    y_coordinates = []
    length_list = len(point_list)


    for i in range(0, length_list):
        x_coordinates.append(point_list[i][0])
        y_coordinates.append(point_list[i][1])
    all_pairs_position = group_to_pairs(x_coordinates, y_coordinates, length_list)


    x_list = list(round(i,1) for i in np.arange(0.1, floor(max(x_coordinates))+0.1, T_l))
    if x_list[-1] != max(x_coordinates) - 0.1:
        x_list.append(max(x_coordinates) - 0.1)


    np_point_list = np.array(point_list)
    mid_point_list = np.array([])
    x_ignore_list = []
    for x_index in x_list:
        endpoint = find_endpoint(x_list[x_list.index(x_index)],all_pairs_position)
        if len(endpoint['y']) == 2:
            if VERBOSE:
                print("Accept: {}".format(x_index))
            mid_point_list = np.append(mid_point_list, (x_index,sum(endpoint['y'].keys())/2))
            #print("{}\n=====".format(endpoint))
        else:
            if VERBOSE:
                print("Ignore: {}".format(x_index))
            x_ignore_list.append(x_index)
    mid_point_x = mid_point_list[0::2]
    mid_point_y = mid_point_list[1::2]

    distance_array = np.array([])
    for mid_point_iter in range(int(mid_point_list.size /2) -1):
        distance_array = np.append(distance_array, distance((mid_point_x[mid_point_iter], mid_point_y[mid_point_iter]),
        (mid_point_x[mid_point_iter+1], mid_point_y[mid_point_iter+1])))
    final_width = []
    for mid_point_iter in range(len(mid_point_x)):
        final_distance_at_1_midpoint = []
        for angle in [30, 60, 90, 120, 150]:
            #print((mid_point_x[mid_point_iter],mid_point_y[mid_point_iter]), angle)
            if angle == 90:
                line_angle_0 = find_endpoint(mid_point_x[mid_point_iter],all_pairs_position)['y'].keys()
                final_distance_at_1_midpoint.append(max(line_angle_0) - min(line_angle_0))
            else:
                test_line = findLine((mid_point_x[mid_point_iter],mid_point_y[mid_point_iter]),angle)
                if VERBOSE:
                    print("At midpoint {}, angle {} -> line: y={:.3f}x+{:.3f}".format((mid_point_x[mid_point_iter],mid_point_y[mid_point_iter]),angle,test_line[0],test_line[1]))
                vertical_list = all_pairs_position['VERTICAL']
                horizontal_list = all_pairs_position['HORIZONTAL']
                line_compare_list_x = {"point":[], "position":[]}
                line_compare_list_y = {"point":[], "position":[]}
            
                for i in all_pairs_position['HORIZONTAL'].items():
                    for j in i[1]:
                        compare_value = (i[0]-test_line[1])/test_line[0]
                        if(in_range(compare_value,j)):
                            line_compare_list_x['point'].append((compare_value, i[0]))
                            line_compare_list_x['position'].append(i[0] >= mid_point_y[mid_point_iter])
                for i in all_pairs_position['VERTICAL'].items():
                    for j in i[1]:
                        compare_value = test_line[0] * i[0] + test_line[1]
                        if(in_range(compare_value,j)):
                            line_compare_list_y['point'].append((i[0],compare_value))
                            line_compare_list_y['position'].append(compare_value >= mid_point_y[mid_point_iter])
                line_compare_combined = {'point':line_compare_list_x['point'] + line_compare_list_y['point'],'position':line_compare_list_x['position'] + line_compare_list_y['position']}
                line_compare_combined['distance'] = list(distance((mid_point_x[mid_point_iter],mid_point_y[mid_point_iter]),i) for i in line_compare_combined['point'])
                #if mid_point_x[mid_point_iter] == 570.1:
                if VERBOSE:
                    print(line_compare_combined)
                above_list = []
                below_list = []
                j = 0
                for i in line_compare_combined['position']:
                    if i:
                        above_list.append(line_compare_combined['distance'][j])
                    else:
                        below_list.append(line_compare_combined['distance'][j])
                    j = j + 1
                final_distance_at_1_midpoint.append(min(above_list)
                + min(below_list))
        final_width.append(min(final_distance_at_1_midpoint))
elapsed_time = time.time() - start
LOG.write("Time with NO Convex Hull:{}[ms]\n".format(elapsed_time/TEST_TIMES * 1000))
LOG.write("Figure size (x,y):{}\n".format((max(x_coordinates), max(y_coordinates)-min(y_coordinates))))
LOG.write("Step T_l:{}\n".format(T_l))
LOG.write("Ignored:{}\n".format(len(x_ignore_list)))
LOG.write("Accepted:{}\n".format(len(mid_point_x)))
LOG.write("Length Measured:{}\n".format(sum(distance_array)))
LOG.write("Width Measured:{}\n".format(sum(final_width)/len(mid_point_x)))

34

In [470]:
final_width_previous = sum(final_width)/len(mid_point_x)
final_length_previous = sum(distance_array)

In [471]:
start = time.time()
for test_times in range(TEST_TIMES):

    processed_points = convexOQHNew.findOrthogonalConvexHull(point_list)
    processed_points.append(processed_points[0])
    processed_points = points_supplement(processed_points)

    x_coordinates = []
    y_coordinates = []
    length_list = len(processed_points)


    for i in range(0, length_list):
        x_coordinates.append(processed_points[i][0])
        y_coordinates.append(processed_points[i][1])
    all_pairs_position = group_to_pairs(x_coordinates, y_coordinates, length_list)


    x_list = list(round(i,1) for i in np.arange(0.1, floor(max(x_coordinates))+0.1, T_l))
    if x_list[-1] != max(x_coordinates) - 0.1:
        x_list.append(max(x_coordinates) - 0.1)


    np_point_list = np.array(processed_points)
    mid_point_list = np.array([])
    x_ignore_list = []
    for x_index in x_list:
        endpoint = find_endpoint(x_list[x_list.index(x_index)],all_pairs_position)
        if len(endpoint['y']) == 2:
            if VERBOSE:
                print("Accept: {}".format(x_index))
            mid_point_list = np.append(mid_point_list, (x_index,sum(endpoint['y'].keys())/2))
            #print("{}\n=====".format(endpoint))
        else:
            if VERBOSE:
                print("Ignore: {}".format(x_index))
            x_ignore_list.append(x_index)
    mid_point_x = mid_point_list[0::2]
    mid_point_y = mid_point_list[1::2]

    distance_array = np.array([])
    for mid_point_iter in range(int(mid_point_list.size /2) -1):
        distance_array = np.append(distance_array, distance((mid_point_x[mid_point_iter], mid_point_y[mid_point_iter]),
        (mid_point_x[mid_point_iter+1], mid_point_y[mid_point_iter+1])))
    final_width = []
    for mid_point_iter in range(len(mid_point_x)):
        final_distance_at_1_midpoint = []
        for angle in [30, 60, 90, 120, 150]:
            #print((mid_point_x[mid_point_iter],mid_point_y[mid_point_iter]))
            if angle == 90:
                line_angle_0 = find_endpoint(mid_point_x[mid_point_iter],all_pairs_position)['y'].keys()
                final_distance_at_1_midpoint.append(max(line_angle_0) - min(line_angle_0))
            else:
                test_line = findLine((mid_point_x[mid_point_iter],mid_point_y[mid_point_iter]),angle)
                if VERBOSE:
                    print("At midpoint {}, angle {} -> line: y={:.3f}x+{:.3f}".format((mid_point_x[mid_point_iter],mid_point_y[mid_point_iter]),angle,test_line[0],test_line[1]))
                vertical_list = all_pairs_position['VERTICAL']
                horizontal_list = all_pairs_position['HORIZONTAL']
                line_compare_list_x = {"point":[], "position":[]}
                line_compare_list_y = {"point":[], "position":[]}
            
                for i in all_pairs_position['HORIZONTAL'].items():
                    for j in i[1]:
                        compare_value = (i[0]-test_line[1])/test_line[0]
                        if(in_range(compare_value,j)):
                            line_compare_list_x['point'].append((compare_value, i[0]))
                            line_compare_list_x['position'].append(i[0] >= mid_point_y[mid_point_iter])
                for i in all_pairs_position['VERTICAL'].items():
                    for j in i[1]:
                        compare_value = test_line[0] * i[0] + test_line[1]
                        if(in_range(compare_value,j)):
                            line_compare_list_y['point'].append((i[0],compare_value))
                            line_compare_list_y['position'].append(compare_value >= mid_point_y[mid_point_iter])
                line_compare_combined = {'point':line_compare_list_x['point'] + line_compare_list_y['point'],'position':line_compare_list_x['position'] + line_compare_list_y['position']}
                line_compare_combined['distance'] = list(distance((mid_point_x[mid_point_iter],mid_point_y[mid_point_iter]),i) for i in line_compare_combined['point'])
                if VERBOSE:
                    print(line_compare_combined)
                above_list = []
                below_list = []
                j = 0
                for i in line_compare_combined['position']:
                    if i:
                        above_list.append(line_compare_combined['distance'][j])
                    else:
                        below_list.append(line_compare_combined['distance'][j])
                    j = j + 1
                final_distance_at_1_midpoint.append(min(above_list) + min(below_list))
        final_width.append(min(final_distance_at_1_midpoint))
elapsed_time = time.time() - start
LOG.write("-----------------\n")
LOG.write("Time with Convex Hull : {}[ms]\n".format(elapsed_time/TEST_TIMES * 1000))
LOG.write("Time with NO Convex Hull:{}[ms]\n".format(elapsed_time/TEST_TIMES * 1000))
LOG.write("Figure size (x,y):{}\n".format((max(x_coordinates), max(y_coordinates)-min(y_coordinates))))
LOG.write("Step T_l:{}\n".format(T_l))
LOG.write("Ignored:{}\n".format(len(x_ignore_list)))
LOG.write("Accepted:{}\n".format(len(mid_point_x)))
LOG.write("Length Measured:{}\n".format(sum(distance_array)))
LOG.write("Width Measured:{}\n".format(sum(final_width)/len(mid_point_x)))

34

In [472]:
# for i in x_ignore_list:
#     endpoint = find_endpoint(x_list[x_list.index(i)],all_pairs_position)
#     print("At x=",i,": lenx=",len(endpoint['x']),", leny=", len(endpoint['y']))

In [473]:
# print(mid_point_x)

In [474]:
# #plt.fill_between(point_list[0], point_list[1],)
# plt.fill_between(np_point_list[:,0],np_point_list[:,1],color='#66e3ff')
# plt.plot(mid_point_x,mid_point_y, c='red')
# plt.xticks(np.arange(0, max(mid_point_x+2), step=1))
# #plt.xlim([10,15]) # zoom in
# plt.show()


In [475]:
# print("Figure size (x,y):", (max(x_coordinates), max(y_coordinates)-min(y_coordinates)))
# print("Step T_l:",T_l)
# print("Ignored:", len(x_ignore_list))
# print("Accepted:", len(mid_point_x))
# print("Final Length:",sum(distance_array))
# print("Final Width:", sum(final_width)/len(mid_point_x))

In [476]:
final_width_new = sum(final_width)/len(mid_point_x)
final_length_new = sum(distance_array)
LOG.write("->Error of Length:{:.1f}%\n".format( ((final_length_new-final_length_previous)/final_length_previous) * 100))
LOG.write("->Error of Width:{:.1f}%\n".format( ((final_width_new-final_width_previous)/final_width_previous)* 100))
LOG.write("\n +++++++++++++++++++++++++++++++\n")
LOG.close()